**Ethical Reflection** </br></br>
The *Ethical Reflection module* is designed to help users evaluate competing approaches to achieving fairness. Users are guided through a series of questions and then prompted to decide on how to respond, along with the ethical reasoning that corresponds to one of three normative ethical theories: outcome-based, duty-based, or character-based. The user is then prompted to examine the consequences of their choices and compare them with alternative approaches. This develops the practical skill of making defensible, ethical decisions using structured reasoning frameworks aligned with normative ethical theories rather than relying on intuition alone.

In [ ]:
#@title pip install
from IPython.display import clear_output

!pip install ipywidgets
clear_output()  # Hides the output after the cell runs

In [ ]:
#@title Imports
import random
import time
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Layout, RadioButtons, Textarea

In [ ]:
#@title Scenarios
scenarios = [
    {
        "title": "Medical AI Accuracy Gap",
        "description": "Your diagnostic AI shows 92% accuracy for white patients and 78% accuracy for Black patients. Hospital wants to deploy next month. Patients need diagnostic help now.",
        "decision_options": [
            "Deploy immediately with current accuracy levels",
            "Delay deployment for 6 months to improve accuracy for all groups",
            "Deploy with prominent warnings about accuracy differences",
            "Deploy only for white patients until accuracy is fixed"
        ],
        "stakeholders": ["Patients needing diagnosis", "Hospital administration", "Medical ethics board", "AI development team", "Community advocates"],
        "consequences": {
            "Deploy immediately with current accuracy levels": {
                "short_term": "Immediate diagnostic help for all patients, but worse outcomes for Black patients",
                "long_term": "Trust erosion in medical AI among minority communities, potential lawsuits",
                "stakeholder_reactions": "Hospital happy, ethics board concerned, community advocates upset"
            },
            "Delay deployment for 6 months to improve accuracy for all groups": {
                "short_term": "No patients get AI assistance, delayed diagnoses for everyone",
                "long_term": "Better long-term trust and accuracy, but some patients harmed by delayed care",
                "stakeholder_reactions": "Patients frustrated, ethics board supportive, hospital loses revenue"
            },
            "Deploy with prominent warnings about accuracy differences": {
                "short_term": "Transparency about limitations, doctors can adjust reliance accordingly",
                "long_term": "Builds trust through honesty, gradual improvement while helping patients",
                "stakeholder_reactions": "Mixed reactions, ethics board cautiously supportive"
            },
            "Deploy only for white patients until accuracy is fixed": {
                "short_term": "Good outcomes for white patients, no help for Black patients",
                "long_term": "Severe trust damage, discrimination lawsuits, public relations disaster",
                "stakeholder_reactions": "Community advocates outraged, hospital faces legal action"
            }
        }
    },
    {
        "title": "Hiring Algorithm Trade-offs",
        "description": "Your hiring AI is 85% accurate overall but shows accuracy gaps: 90% for men, 80% for women. Switching to demographic parity reduces overall accuracy to 75% but equalizes performance.",
        "decision_options": [
            "Keep current algorithm (higher overall accuracy, gender gap)",
            "Switch to demographic parity version (lower accuracy, no gap)",
            "Use hybrid approach with human review for borderline cases",
            "Abandon AI hiring and return to human-only decisions"
        ],
        "stakeholders": ["Job candidates", "Hiring managers", "Company leadership", "Legal team", "Shareholders"],
        "consequences": {
            "Keep current algorithm (higher overall accuracy, gender gap)": {
                "short_term": "Better qualified hires overall, but fewer women hired",
                "long_term": "Legal liability, reputation damage, talent pipeline problems",
                "stakeholder_reactions": "Legal team worried, shareholders focused on performance"
            },
            "Switch to demographic parity version (lower accuracy, no gap)": {
                "short_term": "Equal gender representation, but some less qualified hires",
                "long_term": "Better diversity outcomes, reduced legal risk, possible performance issues",
                "stakeholder_reactions": "Legal team relieved, hiring managers concerned about quality"
            },
            "Use hybrid approach with human review for borderline cases": {
                "short_term": "Balanced approach but higher costs and slower hiring",
                "long_term": "Good compromise but scalability issues and inconsistent outcomes",
                "stakeholder_reactions": "Most stakeholders cautiously supportive but concerned about costs"
            },
            "Abandon AI hiring and return to human-only decisions": {
                "short_term": "No algorithmic bias but return to human bias and inefficiency",
                "long_term": "Slower hiring, potential for other forms of bias, competitive disadvantage",
                "stakeholder_reactions": "Mixed feelings, some relief about avoiding AI bias"
            }
        }
    }
]

ethical_frameworks = {
    "Outcome-based": {
        "description": "Focus on consequences and results. What decision produces the best outcomes for the most people?",
        "questions": [
            {
                "question": "Who would benefit MOST from your decision?",
                "type": "radio",
                "options": ["Patients/Users", "Organization", "Society overall", "Specific disadvantaged groups"]
            },
            {
                "question": "What's your biggest concern about long-term consequences?",
                "type": "radio",
                "options": ["Trust erosion", "Legal liability", "Competitive disadvantage", "Ongoing harm to people"]
            },
            {
                "question": "How do you weigh immediate benefits vs. future costs?",
                "type": "radio",
                "options": ["Immediate benefits more important", "Future costs more important", "Try to balance both", "Focus on minimizing worst outcomes"]
            }
        ]
    },
    "Duty-based": {
        "description": "Focus on rights, duties, and moral principles. What decision respects fundamental obligations regardless of outcomes?",
        "questions": [
            {
                "question": "What fundamental right is most at stake here?",
                "type": "radio",
                "options": ["Equal treatment", "Right to healthcare/services", "Right to transparency", "Right to dignity"]
            },
            {
                "question": "What is your most important moral obligation?",
                "type": "radio",
                "options": ["Do no harm", "Treat everyone equally", "Be honest and transparent", "Respect human autonomy"]
            },
            {
                "question": "Could your decision become a universal rule that everyone should follow?",
                "type": "radio",
                "options": ["Yes, this should be the standard approach", "Yes, but only in similar situations", "No, this is a special case", "Unsure - depends on context"]
            }
        ]
    },
    "Character-based": {
        "description": "Focus on virtues and character. What decision reflects the character traits of a just and prudent organization?",
        "questions": [
            {
                "question": "What virtue is most important in this situation?",
                "type": "radio",
                "options": ["Justice (what's deserved)", "Prudence (wise judgment)", "Honesty (transparency)", "Compassion (care for others)"]
            },
            {
                "question": "What would a morally exemplary organization do?",
                "type": "radio",
                "options": ["Prioritize fairness above all", "Make the most responsible long-term choice", "Be completely transparent about limitations", "Focus on helping those most in need"]
            },
            {
                "question": "What character trait will this decision reinforce in your organization?",
                "type": "radio",
                "options": ["Commitment to equality", "Practical wisdom", "Intellectual honesty", "Profit-focused pragmatism"]
            }
        ]
    }
}

In [ ]:
#@title Decisions
class EthicalDecisionNavigator:
    def __init__(self, scenarios_list, frameworks_dict):
        self.scenarios = scenarios_list.copy()
        self.frameworks = frameworks_dict
        self.current_scenario = 0
        self.decisions_made = []

        random.shuffle(self.scenarios)

        # Create widgets
        self.scenario_display = HTML(value="<h3>Ethical Decision Navigator</h3>")

        self.decision_options = RadioButtons(
            options=[],
            description='Decision:',
            disabled=False,
            layout=Layout(width='100%')
        )

        self.framework_selector = RadioButtons(
            options=list(self.frameworks.keys()),
            description='Reasoning Approach:',
            disabled=False,
            layout=Layout(width='100%')
        )

        self.framework_guidance = HTML(value="")

        self.reflection_questions = VBox([])

        self.submit_button = Button(
            description='Make Decision',
            button_style='warning',
            layout=Layout(width='200px', height='40px', margin='10px')
        )

        self.consequences_display = HTML(value="")

        self.alternatives_display = HTML(value="")

        self.next_button = Button(
            description='Next Scenario →',
            button_style='',
            layout=Layout(width='150px', height='30px', margin='10px')
        )

        self.progress_display = HTML(value="")

        # Button clicks
        self.framework_selector.observe(self.show_framework_guidance, names='value')
        self.submit_button.on_click(lambda b: self.process_decision())
        self.next_button.on_click(lambda b: self.next_scenario())

        # Layout
        submit_box = HBox([self.submit_button], layout=Layout(justify_content='center'))
        next_box = HBox([self.next_button], layout=Layout(justify_content='center'))

        self.widget = VBox([
            self.scenario_display,
            self.decision_options,
            self.framework_selector,
            self.framework_guidance,
            self.reflection_questions,
            submit_box,
            self.consequences_display,
            self.alternatives_display,
            next_box,
            self.progress_display
        ])

    def start_navigation(self):
        self.current_scenario = 0
        self.decisions_made = []
        self.show_scenario()
        return self.widget

    def show_scenario(self):
        if self.current_scenario >= len(self.scenarios):
            self.show_final_analysis()
            return

        scenario = self.scenarios[self.current_scenario]

        # Reset interface
        self.decision_options.options = scenario['decision_options']
        self.decision_options.value = None
        self.framework_selector.value = list(self.frameworks.keys())[0]
        self.consequences_display.value = ""
        self.alternatives_display.value = ""
        self.next_button.layout.display = 'none'

        self.scenario_display.value = f"""
        <div style='padding: 20px; background: #f9f9f9; border-radius: 10px; margin: 10px 0; text-align: center;'>
            <h3>Scenario {self.current_scenario + 1}: {scenario['title']}</h3>
            <p style='font-size: 16px; line-height: 1.5;'><strong>{scenario['description']}</strong></p>
            <p style='color: #666; margin-top: 15px;'><em><strong>⚖️ Decision Required:</strong> Choose one option below, select your reasoning approach, and answer the guided questions to make a defensible ethical decision.</em></p>
        </div>
        """

        self.show_framework_guidance({'new': self.framework_selector.value})
        self.update_progress()

    def show_framework_guidance(self, change):
        framework = change['new']
        if framework in self.frameworks:
            framework_info = self.frameworks[framework]

            self.framework_guidance.value = f"""
            <div style='border: 2px solid #3498DB; padding: 15px; border-radius: 8px; margin: 10px 0; background: #EBF3FD;'>
                <h4 style='color: #2980B9; margin: 0;'>🧭 {framework} Reasoning</h4>
                <p><strong>Approach:</strong> {framework_info['description']}</p>
            </div>
            """

            # Create reflection questions
            self.create_reflection_questions(framework_info['questions'])

    def create_reflection_questions(self, questions):
        question_widgets = []

        for i, q in enumerate(questions):
            question_html = HTML(value=f"<p style='font-weight: bold; margin: 10px 0 5px 0;'>{i+1}. {q['question']}</p>")

            if q['type'] == 'radio':
                answer_widget = RadioButtons(
                    options=q['options'],
                    disabled=False,
                    layout=Layout(width='100%', margin='0 0 15px 20px')
                )
                answer_widget.question_id = i

            question_widgets.extend([question_html, answer_widget])

        self.reflection_questions.children = question_widgets

    def get_reflection_answers(self):
        answers = {}
        widgets = self.reflection_questions.children

        for widget in widgets:
            if hasattr(widget, 'question_id') and hasattr(widget, 'value'):
                if widget.value:  # Only include answered questions
                    answers[widget.question_id] = widget.value

        return answers

    def process_decision(self):
        if not self.decision_options.value:
            self.consequences_display.value = '''
            <div style="border: 2px solid orange; padding: 15px; border-radius: 8px; background: #fff8e1; text-align: center;">
                <p><strong>Please select a decision option first!</strong></p>
            </div>
            '''
            return

        # Check if reflection questions are answered
        answers = self.get_reflection_answers()
        framework = self.framework_selector.value
        expected_questions = len(self.frameworks[framework]['questions'])

        if len(answers) < expected_questions:
            self.consequences_display.value = f'''
            <div style="border: 2px solid orange; padding: 15px; border-radius: 8px; background: #fff8e1; text-align: center;">
                <p><strong>Please answer all {expected_questions} reflection questions!</strong></p>
            </div>
            '''
            return

        scenario = self.scenarios[self.current_scenario]
        decision = self.decision_options.value

        # Record decision
        self.decisions_made.append({
            'scenario': scenario['title'],
            'decision': decision,
            'framework': framework,
            'reflection_answers': answers
        })

        # Show consequences
        consequences = scenario['consequences'][decision]
        self.consequences_display.value = f"""
        <div style='border: 2px solid #E74C3C; padding: 20px; border-radius: 10px; margin: 10px 0; background: #FDEDEC;'>
            <h4 style='color: #C0392B; text-align: center;'>⚡ Consequences of Your Decision</h4>
            <p><strong>Your Choice:</strong> {decision}</p>
            <p><strong>Short-term Impact:</strong> {consequences['short_term']}</p>
            <p><strong>Long-term Impact:</strong> {consequences['long_term']}</p>
            <p><strong>Stakeholder Reactions:</strong> {consequences['stakeholder_reactions']}</p>
        </div>
        """

        # Show how other frameworks would decide
        self.show_alternative_approaches(scenario, decision, framework)

        # Show next button
        self.next_button.layout.display = 'block'
        self.submit_button.disabled = True

    def show_alternative_approaches(self, scenario, chosen_decision, chosen_framework):
        other_frameworks = [f for f in self.frameworks.keys() if f != chosen_framework]

        alternatives_html = f"""
        <div style='border: 2px solid #9B59B6; padding: 20px; border-radius: 10px; margin: 10px 0; background: #F4F1F8;'>
            <h4 style='color: #8E44AD; text-align: center;'>🔄 How Other Ethical Approaches Would Decide</h4>
        """

        # Example likely choices for each framework
        framework_preferences = {
            "Medical AI Accuracy Gap": {
                "Outcome-based": "Deploy with prominent warnings about accuracy differences",
                "Duty-based": "Delay deployment for 6 months to improve accuracy for all groups",
                "Character-based": "Deploy with prominent warnings about accuracy differences"
            },
            "Hiring Algorithm Trade-offs": {
                "Outcome-based": "Use hybrid approach with human review for borderline cases",
                "Duty-based": "Switch to demographic parity version (lower accuracy, no gap)",
                "Character-based": "Use hybrid approach with human review for borderline cases"
            }
        }

        scenario_prefs = framework_preferences.get(scenario['title'], {})

        for framework in other_frameworks:
            framework_info = self.frameworks[framework]
            likely_choice = scenario_prefs.get(framework, scenario['decision_options'][0])

            alternatives_html += f"""
            <div style='margin: 10px 0; padding: 10px; background: white; border-radius: 5px;'>
                <p><strong>{framework} Approach:</strong> {framework_info['description']}</p>
                <p><strong>Likely Decision:</strong> {likely_choice}</p>
            </div>
            """

        alternatives_html += """
        <p style='background: #FFFFCC; padding: 10px; border-radius: 5px; font-style: italic; margin-top: 15px;'>
        <strong>💡 Insight:</strong> Notice how different ethical frameworks can lead to different decisions for the same situation.
        The guided questions help you work through the reasoning process systematically.
        </p>
        </div>
        """

        self.alternatives_display.value = alternatives_html

    def next_scenario(self):
        self.current_scenario += 1
        self.submit_button.disabled = False
        self.show_scenario()

    def update_progress(self):
        total_scenarios = len(self.scenarios)

        self.progress_display.value = f"""
        <div style='background: #e8f4fd; padding: 10px; border-radius: 5px; text-align: center;'>
            <strong>Progress:</strong> Scenario {self.current_scenario + 1}/{total_scenarios} |
            <strong>Decisions Made:</strong> {len(self.decisions_made)}
        </div>
        """

    def show_final_analysis(self):
        # Analyze decision patterns
        framework_usage = {}
        for decision in self.decisions_made:
            framework = decision['framework']
            framework_usage[framework] = framework_usage.get(framework, 0) + 1

        most_used_framework = max(framework_usage, key=framework_usage.get) if framework_usage else "None"

        self.scenario_display.value = f"""
        <div style='border: 3px solid #4CAF50; padding: 25px; border-radius: 15px; background: #f8fff8; text-align: center;'>
            <h2 style='color: #4CAF50;'>Ethical Decision Navigation Complete! ⚖️</h2>

            <h3>Your Decision-Making Pattern</h3>
            <p><strong>Most Used Framework:</strong> {most_used_framework}</p>
            <p><strong>Framework Distribution:</strong> {dict(framework_usage)}</p>

            <h3>Developed Skills</h3>
            <ul style='text-align: left; max-width: 600px; margin: 0 auto;'>
                <li><strong>Structured Decision-Making:</strong> Used systematic ethical frameworks</li>
                <li><strong>Guided Reflection:</strong> Worked through key reasoning questions</li>
                <li><strong>Consequence Awareness:</strong> Saw real impacts of your choices</li>
                <li><strong>Framework Flexibility:</strong> Experienced different approaches to ethics</li>
            </ul>

            <h4>Key Insight: Systematic Ethical Reasoning</h4>
            <p>You've practiced making defensible ethical decisions by answering guided questions that lead you through
            proper reasoning patterns. This structured approach builds the thinking skills needed for real-world fairness challenges.</p>
        </div>
        """

        # Hide other elements
        self.decision_options.layout.display = 'none'
        self.framework_selector.layout.display = 'none'
        self.framework_guidance.value = ""
        self.reflection_questions.children = []
        self.submit_button.layout.display = 'none'
        self.consequences_display.value = ""
        self.alternatives_display.value = ""
        self.next_button.layout.display = 'none'

In [ ]:
#@title Start the Module
# Start the Ethical Decision Navigator
navigator = EthicalDecisionNavigator(scenarios, ethical_frameworks)
widget = navigator.start_navigation()
display(widget)